In [1]:
# importa as bibliotecas necessárias
import PyPDF2
import re
import pandas as pd

In [2]:
# Abre o arquivo pdf 
path = r'C:\Users\MatheusPereira\OneDrive - Pontte\Área de Trabalho\automacaoRegistroCCI\Contratos\HE_Contrato_Agostinho_Assinatura Digital_VFinal.pdf'
pdf_file = open(path, 'rb')


In [3]:
#Faz a leitura usando a biblioteca
read_pdf = PyPDF2.PdfFileReader(pdf_file)

# pega o numero de páginas
number_of_pages = read_pdf.getNumPages()

#Extriar Texto Página 1 a 5
text=''
for i in range(0,4):
    #Ler Página PDF
    pageObj = read_pdf.getPage(i)
    #Extrair Texto
    text=text+pageObj.extractText()

#Tratar Texto (Remover Quebra de Linhas)
text = re.sub('\r', '', text) 
text = re.sub('\n', '', text)

In [4]:
text

'CONTRATO DE EMPRÉSTIMO (MÚTUO) E PACTO ADJETO DE ALIENAÇÃO FIDUCIÁRIA EM GARANTIA  COM EMISSÃO DE CÉDULA DE CRÉDITO IMOBILIÁRIO  - CCI  CONTRATO Nº 0001013447/AFD REFIN – SFI  RUBRICA DAS PARTES  Contrato de Empréstimo (Mútuo) com Alienação Fiduciária – Pessoa Física Página 1 de 36   Pelo presente contrato, celebrado nos termos do artigo 38 da Lei 9.514/97, de acordo com a redação  que  lhe  deu  o  artigo  57  da  Lei  10.931/2004, as  partes  a  seguir  enunciadas  e qualificadas  formalizam  este  CONTRATO  DE EMPRÉSTIMO (MÚTUO)  E  PACTO ADJETO DE ALIENAÇÃO FIDUCIÁRIA EM GARANTIA  COM EMISSÃO DE CÉDULA DE CRÉDITO IMOBILIÁRIO - CCI, tendo por objeto o imóvel identificado no campo 5 a seguir, e consoante cláusulas e condições adiante estipuladas :  CAMPO 1 – CREDORA RAZÃO SOCIAL: QI SOCIEDADE DE CRÉDITO DIRETO S.A.    ENDEREÇO COMERCIAL: Avenida Brigadeiro Faria Lima, nº 2.391, 1º andar  COMPLEMENTO: Conjunto 12 BAIRRO: Jardim Paulistano CIDADE: São Paulo UF: SP   CEP: 01452-000  CN

In [9]:
listaDePara = {'valorTotal':'VALOR DO EMPRÉSTIMO: R$','tabela': 'SISTEMA DE AMORTIZAÇÃO:','registro': 'DESPESAS DE REGISTRO: R$','Taxa ao Mes':'H.1. NOMINAL:','valorLiquido': '-J-K-L-M-N): R$',
                'prazoMes': 'PRAZO DE AMORTIZAÇÃO:','valorPrimeiraParcelaComEncargos':'VALOR TOTAL DO PRIMEIRO ENCARGO, NESTA DATA:  R$',
                'valorImóvel':'Valor de avaliação do Imóvel para fins de leilão:  R$','prazoContrato': 'N.º DE PRESTAÇÕES:','ultimaParcela':'DATA DO TÉRMINO DO PRAZO CONTRATUAL: ','dataContrato': 'DATA DE DESEMBOLSO:',
                'valorPrimeiraParcela':'VALOR DA PARCELA MENSAL (AMORTIZAÇÃO E JUROS), NESTA DATA:  R$','primeiraParcela':'VENCIMENTO DA PRIMEIRA PRESTAÇÃO :','indice':'ÍNDICE: MENSAL do'
                }

#len(listaHE) # <--- Qtd de Itens na Lista
listaKey = []
listaValues = []

for key, value in listaDePara.items():
    inicioFrase = text.find(value,0)
    finalFrase = inicioFrase + len(value) + 1
    proximoEspaco = text.find(" ", finalFrase)
    valorExtraido = text[finalFrase:proximoEspaco]

    #Ajustar Valores Númericos
    if '.' in valorExtraido:
        valorExtraido = valorExtraido.replace(".", "")
        valorExtraido = valorExtraido.replace(",", ".")
    
    #Ajustar Valores Percentuais
    if '%' in valorExtraido:
        valorExtraido = valorExtraido.replace(",", ".")
        valorExtraido = valorExtraido.replace("%", "")
        valorExtraido = round(float(valorExtraido)/100,4)
    elif 'IPCA' or 'IGPM' in valorExtraido:
        valorExtraido = valorExtraido.replace(",", "")
    
    listaKey.append(key)
    listaValues.append(valorExtraido)

In [10]:
# Extraindo numero da matricula
inicioFrase = text.find('matrícula nº',0)
finalFrase = inicioFrase + len('matrícula nº') + 1
ultimaMatricula = text.find("do", finalFrase)
valorExtraido = text[finalFrase:ultimaMatricula]
valorExtraido = valorExtraido.replace(",", "")
if len(valorExtraido) > 9:
    valorExtraido = 0
    inicioFrase = text.find('nas matrículas nºs',0)
    finalFrase = inicioFrase + len('nas matrículas nºs') + 1
    ultimaMatricula = text.find("do", finalFrase)
    valorExtraido = text[finalFrase:ultimaMatricula]
    valorExtraido = valorExtraido.replace(",", "")
    valorExtraido = valorExtraido.replace("e ", "")

listaKey.append('Matrícula')
listaValues.append(valorExtraido)
valorExtraido

'141.951 '

In [11]:
# Extraindo cartório
inicioFrase = text.find(' do Livro N°',0)
finalFrase = inicioFrase + len(' do Livro N°') + 1
ultimaMatricula = text.find(".", finalFrase)
valorExtraido = text[finalFrase:ultimaMatricula]

listaKey.append('Cartório')
listaValues.append(valorExtraido)

In [12]:
#Criar Dicionario das duas Listas
dict_keyValue = dict(zip(listaKey,listaValues))
dict_keyValue

{'valorTotal': '185236.16',
 'tabela': 'SAC',
 'registro': '3042.67',
 'Taxa ao Mes': 0.1162,
 'valorLiquido': '169200.00',
 'prazoMes': '240',
 'valorPrimeiraParcelaComEncargos': '2754.90',
 'valorImóvel': '376000.00',
 'prazoContrato': '234',
 'ultimaParcela': '30/06/2042',
 'dataContrato': '08/06/2022',
 'valorPrimeiraParcela': '2662.09',
 'primeiraParcela': '30/01/2023',
 'indice': 'IPCA',
 'Matrícula': '141.951 ',
 'Cartório': '02 de Registro Geral do 3º Oficial de Registro de Imóveis da Comarca de  São Paulo/SP'}